# This notebook creates a test tsv file for submitting to the Azure ML endpoint.

In [5]:
library('uuid')
library(jsonlite)
library(plyr)
library('geosphere')

Loading required package: sp


In [6]:
#Set the locations and calculate the distance matrix for the possible GPS locations of the hands.
locations <- rbind(c(-115.171579,36.101551), c(-115.168599,36.104740), c(-115.168599,36.104740), c(-115.175448,36.098796),
                   c(-115.166224,36.125926), c(-115.172305, 36.124703), c(-115.138972, 36.168953),
                   c(-115.145404, 36.170495), c(-115.147113, 36.171535), c(-115.145318, 36.174308),
                   c(-115.145038, 36.172837), c(-115.151643, 36.135330), c(-115.174947, 36.120970),
                   c(-115.176478, 36.121561), c(-74.435764,39.356116 ))
location_names <- c('MGM Grand Show Bar','MGM Grand Garden Area', 'New York/New York', 'Exalibur', 'Wynn', 'Treasure Island','El Cortez', 
                    'Golden Nugget', 'Plaza', 'Main Street Station', 'California', 'Westgate',
                    'Mirage Players Club','Mirage Calypso', 'Caesars')
location_sub <- c('R1T1H1','R1T1H2','R1T2H1','R1T2H2','R2T1H1','R2T1H2','R2T2H1','R2T2H2','R3T1H1','R3T1H2','R3T2H1','R3T2H2')
#print(locations[8,])
#print(location_names[[8]])
distances<-distm(locations)/1000
#distances
all_locations<-merge(data.frame(location_names,stringsAsFactors=FALSE),data.frame(location_sub,stringsAsFactors=FALSE))
names(all_locations)<-c("Loc.Name","Loc.Sub")
#all_locations <- ddply(locations[grep(hand.loc.name, location_names),2]
                           
all_locations<-ddply(all_locations,c("Loc.Name"),
      transform,
      Loc.lat = locations[grep(Loc.Name[[1]], location_names),2],
      Loc.lon = locations[grep(Loc.Name[[1]], location_names),1])
                           

In [8]:
#Number of hands to generate
nhands <- 3

#Generate deck for creating hands
deck <- paste0(rep(c(2:9, "T", "J", "Q", "K", "A"), 4),  #card values
               rep(c("s", "h", "d", "c"), each = 13)) #suits

#The Card information comes from a JSON array with the following information:
#* FaceValue: 2 character string
#* CardID: a UUID for each card
#* HandID: (starting HandID for each card)
#* Hand.Loc.Lat (start location latitude)
#* Hand.Loc.Lon: (start location longitude)
#* Hand.Loc.Name: (start Casino name)
#* Hand.Loc.Sub: (start table, hand, seat ID for each room)
#* TransportMult:  (multiplier on moving this card (mostly 1))
#* OccupiedLevel: (0 = available, 1000 = can't move (i.e. table card in Hold'em)

hand_locations <- all_locations[sample(nrow(all_locations), nhands), ]

cardDF <- data.frame(FaceValue = character(), 
                        CardID=character(),
                        HandID=character(),
                        HandLocName=character(),
                        HandLocSub=character(),
                        HandLocLat=numeric(),
                        HandLocLon=numeric(),
                        CardTransMult=numeric(),
                        CardOccLevel = numeric(),stringsAsFactors=FALSE)

for (handnum in 1:nhands){
    shuffledhand <-sample(deck,size=5) #Shuffle decks
    
    handlocname<-hand_locations[handnum,1]
    handlocsub <-hand_locations[handnum,2]
    #print(hand.loc.name)
    #Will need hand.loc index to get distance matrix: grep(hand.loc.name, location_names)
    handloclat <- locations[grep(handlocname, location_names),2]
    handloclon <- locations[grep(handlocname, location_names),1]
    handID = UUIDgenerate() #unique handID number
    for (i in 1:length(shuffledhand)){
        #simulate bringing in JSON data on each card (from the Dealer server for now)
        
        cardString<-toJSON(data.frame(FaceValue=shuffledhand[[i]],
                        CardID=UUIDgenerate(),
                        HandID=handID,
                        HandLocName=handlocname,
                        HandLocSub=handlocsub,
                        HandLocLat=handloclat,
                        HandLocLon=handloclon,
                        CardTransMult=runif(1)*(1+rpois(1, 0.5)),
                        CardOccLevel = 1000 * rpois(1,0.1)))
        #Join all the cards to the main data frame
        
        
        
        cardDF <- rbind(cardDF, fromJSON(cardString))
    }
}
batchID<-UUIDgenerate()

cardData<-toJSON(cardDF)
batchDF<-data.frame(batchRequestTime=strftime(as.POSIXlt(Sys.time(), "UTC"), "%Y-%m-%dT%H:%M:%S%z"),
                        batchID=batchID,
                        cardData=as.character(cardData),stringsAsFactors=FALSE)


write.table(batchDF, file = "handFormat.tsv", quote = TRUE, sep = "\t",
            eol = "\n", na = "NA", dec = ".", row.names = FALSE,
            col.names = TRUE, qmethod = c("escape", "double"))


In [18]:
cardDF

FaceValue,CardID,HandID,HandLocName,HandLocSub,HandLocLat,HandLocLon,CardTransMult,CardOccLevel
7s,70da3932-a76e-11e6-822a-a12bcfd83528,70da25a0-a76e-11e6-822a-a12bcfd83528,Mirage Players Club,R3T2H2,36.1210,-115.1749,0.2574,0
Ac,70dad572-a76e-11e6-822a-a12bcfd83528,70da25a0-a76e-11e6-822a-a12bcfd83528,Mirage Players Club,R3T2H2,36.1210,-115.1749,0.4674,1000
Jc,70db5e34-a76e-11e6-822a-a12bcfd83528,70da25a0-a76e-11e6-822a-a12bcfd83528,Mirage Players Club,R3T2H2,36.1210,-115.1749,0.2210,0
5c,70dbfa60-a76e-11e6-822a-a12bcfd83528,70da25a0-a76e-11e6-822a-a12bcfd83528,Mirage Players Club,R3T2H2,36.1210,-115.1749,0.8325,0
5d,70dc6f86-a76e-11e6-822a-a12bcfd83528,70da25a0-a76e-11e6-822a-a12bcfd83528,Mirage Players Club,R3T2H2,36.1210,-115.1749,1.6170,1000
4h,70dce4b6-a76e-11e6-822a-a12bcfd83528,70dcd12e-a76e-11e6-822a-a12bcfd83528,Exalibur,R3T1H1,36.0988,-115.1754,0.6376,0
Ad,70dd465e-a76e-11e6-822a-a12bcfd83528,70dcd12e-a76e-11e6-822a-a12bcfd83528,Exalibur,R3T1H1,36.0988,-115.1754,0.1711,0
3s,70ddbb98-a76e-11e6-822a-a12bcfd83528,70dcd12e-a76e-11e6-822a-a12bcfd83528,Exalibur,R3T1H1,36.0988,-115.1754,1.0124,0
Tc,70de57c4-a76e-11e6-822a-a12bcfd83528,70dcd12e-a76e-11e6-822a-a12bcfd83528,Exalibur,R3T1H1,36.0988,-115.1754,0.8605,0
Kc,70decd08-a76e-11e6-822a-a12bcfd83528,70dcd12e-a76e-11e6-822a-a12bcfd83528,Exalibur,R3T1H1,36.0988,-115.1754,0.0785,0
